In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [12]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [11]:
DELETE_TABLES=True
if DELETE_TABLES:
    try:
        cmd="""DROP TABLE `%s`"""%osm_intersections_table_name
        cursor.execute(cmd)
    except Exception as e:
        print(e)
        pass
    cmd="""DROP TABLE `%s`"""%osm_nodes_table_name
    cursor.execute(cmd)
2

Unread result found


InternalError: Unread result found

In [4]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `osm_id` BIGINT NOT NULL,
    `contour_line_id` INT NOT NULL,
    `elevation` FLOAT,

    FOREIGN KEY (contour_line_id) REFERENCES %s(id),
    PRIMARY KEY (osm_id)
    );
        """%(osm_nodes_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)



    
    cmd="""CREATE TABLE `%s`(
        `osm_begin` BIGINT NOT NULL,
        `osm_end` BIGINT NOT NULL,
        `osm_key` INT NOT NULL,
        `contour_line_id` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (osm_begin) REFERENCES %s(osm_id),
        FOREIGN KEY (osm_end) REFERENCES %s(osm_id),
        FOREIGN KEY (contour_line_id) REFERENCES %s(id),
        PRIMARY KEY (osm_begin,osm_end,osm_key,edge_coordinate)
        

          )
          ENGINE=InnoDB;
        """%(osm_intersections_table_name,osm_nodes_table_name,osm_nodes_table_name,contours_lines_table_name)
    cursor.execute(cmd)
    

2

2

In [5]:
t=time.time()

In [6]:
t1=time.time()
G=build_graph(cursor)
max_depth=np.max([data['depth'] for _,data in G.nodes(data=True)])
t2=time.time()
print(t2-t1)

0.6856567859649658


In [7]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='all_private')
G_osm=to_multi_graph(G_osm)
osm_crs=ox.graph_to_gdfs(G_osm,edges=False).estimate_utm_crs()
G_osm=ox.project_graph(G_osm,to_crs=osm_crs)
add_missing_geometries(G_osm)
t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

553.6356892585754


(83574, 102644)

In [8]:
t1=time.time()
osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True)
osm_edges_df['edge']=osm_edges_df.index
osm_edges_df=osm_edges_df.loc[:,['edge','geometry','length']]
osm_edges_df['number_of_points']=osm_edges_df['geometry'].apply(lambda ls:len(ls.coords))
osm_edges_df=osm_edges_df.sort_values(by='length')
osm_edges_df=osm_edges_df.set_index(np.arange(len(osm_edges_df)))
t2=time.time()
print(t2-t1)
osm_edges_df

2.9538686275482178


,edge,geometry,length,number_of_points
0,"(3703607370, 3703607388, 0)","LINESTRING (539398.556 4710781.746, 539398.400...",0.156,2
1,"(3694079844, 3694079836, 0)","LINESTRING (539391.686 4710743.675, 539391.464...",0.221,2
2,"(2339806071, 8947372900, 0)","LINESTRING (482028.447 4620777.742, 482028.288...",0.222,2
3,"(5695187353, 6418221376, 0)","LINESTRING (512478.298 4666698.070, 512478.415...",0.332,2
4,"(7825637697, 7825637699, 0)","LINESTRING (516231.708 4611051.794, 516232.108...",0.486,2
...,...,...,...,...
102639,"(4339981243, 10792988043, 0)","LINESTRING (500339.153 4645123.664, 500337.141...",9160.843,197
102640,"(1038302195, 2311673701, 0)","LINESTRING (511142.530 4600708.022, 511071.497...",9741.848,226
102641,"(411225727, 411226080, 0)","LINESTRING (501360.704 4683640.750, 501314.472...",10112.537,358
102642,"(335643757, 3526675295, 0)","LINESTRING (484773.804 4697381.558, 484762.634...",10606.654,470


In [13]:
t1=time.time()
intersection,depth=decreasing_depth_intersections(osm_edges_df,osm_crs,G,cursor,max_depth,depth_step=20,max_delta_time_total=float('inf'),max_delta_time_per_level=float('inf'))
t2=time.time()
print('total : %f'%(t2-t1),depth)

520<depth<=540 :0.365381
500<depth<=520 :0.401291
480<depth<=500 :0.480981
460<depth<=480 :0.587238
440<depth<=460 :0.826297
420<depth<=440 :1.031117
400<depth<=420 :1.429726
380<depth<=400 :1.930458
360<depth<=380 :2.469012
340<depth<=360 :2.929211
320<depth<=340 :3.633399
300<depth<=320 :4.328937
280<depth<=300 :5.279547
260<depth<=280 :6.212221
240<depth<=260 :7.391432
220<depth<=240 :11.414310
200<depth<=220 :14.010498
180<depth<=200 :16.871232
160<depth<=180 :21.994266
140<depth<=160 :25.885676
120<depth<=140 :30.262393
100<depth<=120 :45.649025
80<depth<=100 :61.572524
60<depth<=80 :68.173871
40<depth<=60 :70.274724
20<depth<=40 :81.955644
0<depth<=20 :133.819916
total : 621.190440 1


In [ ]:
intersection['edge_coordinate']=None
t1=time.time()
for edge,df in intersection.groupby('edge'):
    ls=osm_edges_df[osm_edges_df.edge==edge].iloc[0]['geometry']
    df['edge_coordinate']=df['geometry'].apply(lambda pt:ls.project(pt))
    intersection.loc[intersection.edge==edge,'edge_coordinate']=df['edge_coordinate']
t2=time.time()
print(t2-t1)

In [ ]:
t1=time.time()
pre_osm_nodes_to_contours={}
for edge,df in intersection.groupby('edge'):
    df=df.sort_values('edge_coordinate')
    u,v=edge[:2]
    if not u in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[u]={df.iloc[0]['id']}
    else:
        pre_osm_nodes_to_contours[u].add(df.iloc[0]['id'])
    if not v in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[v]={df.iloc[-1]['id']}
    else:
        pre_osm_nodes_to_contours[v].add(df.iloc[-1]['id'])
t2=time.time()
print(t2-t1)

In [ ]:
t1=time.time()
osm_nodes_to_contours={}
ambiguous_osm_nodes_to_contours={}
for osm_node,contour_ids in pre_osm_nodes_to_contours.items():
    if len(contour_ids)>1:
        id_1,id_2=list(contour_ids)[:2]
        if id_2 in G.neighbors(id_1):
            osm_nodes_to_contours[osm_node]=id_1
        else:
            predecessors=list(G.predecessors(id_2))
            contour_id=id_2 if len(predecessors)==0 else predecessors[0]
            osm_nodes_to_contours[osm_node]=contour_id
    else:
        ambiguous_osm_nodes_to_contours[osm_node]=list(contour_ids)[0]
t2=time.time()
print(t2-t1)

In [ ]:
t1=time.time()
ambiguous_contours_df=get_nodes_data(cursor,set(ambiguous_osm_nodes_to_contours.values()))
ambiguous_contours_df=ambiguous_contours_df.set_index('id')
ambiguous_contours_df=ambiguous_contours_df.to_crs(osm_crs)
ambiguous_contours_df['geometry']=ambiguous_contours_df['geometry'].apply(lambda ls:Polygon(ls))
t2=time.time()
print('df ready, took %f'%(t2-t1))
for osm_node,id_contour in ambiguous_osm_nodes_to_contours.items():
    datum=G_osm.nodes()[osm_node]
    pt=Point(datum['x'],datum['y'])
    if pt.within(ambiguous_contours_df.loc[id_contour]['geometry']):
        osm_nodes_to_contours[osm_node]=id_contour
    else:
        predecessors=list(G.predecessors(id_contour))
        id_contour=id_contour if len(predecessors)==0 else predecessors[0]
        osm_nodes_to_contours[osm_node]=id_contour

t3=time.time()
print('ambiguities removed, took %f'%(t3-t2))

In [ ]:
ADD_OSM_DATA=True
if ADD_OSM_DATA:
    t1=time.time()
    string_list=',\n'.join([str((u,v)) for u,v in osm_nodes_to_contours.items()])
    cmd="INSERT INTO %s (`osm_id`,`contour_line_id`) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)

    string_list=',\n'.join(['(%i,%i,%i,%i,%f,%f)'%(row['id'],row['edge'][0],row['edge'][1],row['edge'][2],row['edge_coordinate'],row['elevation'])  for _,row in intersection.iterrows()])

    cmd="INSERT INTO %s (`contour_line_id`,`osm_begin`,`osm_end`,`osm_key`,`edge_coordinate`,`elevation`) VALUES %s;"%(osm_intersections_table_name,string_list)
    cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)

In [ ]:
tt=time.time()
print(tt-t)